In [1]:
import tensorflow as tf

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras import optimizers
from pandas import Series
import math
import numpy
from tensorflow.keras.layers import *
from tensorflow.keras import layers
import pandas as pd
from math import *
from sklearn.metrics import *

data = pd.read_csv("C:\\Users\\mozzimerci\\jupyter notebook\\time\\bike_pm2.5\\PRSA_data_2010.1.1-2014.12.31.csv")

In [2]:
from graphviz import *
from sklearn.datasets import load_iris
from sklearn import tree
 
from collections import Counter
from itertools import chain, combinations
 
import ast
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import graphviz
import seaborn as sns
from sklearn import datasets, svm, metrics
from pandas import DataFrame

In [3]:
data

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,43820,2014,12,31,19,8.0,-23,-2.0,1034.0,NW,231.97,0,0
43820,43821,2014,12,31,20,10.0,-22,-3.0,1034.0,NW,237.78,0,0
43821,43822,2014,12,31,21,10.0,-22,-3.0,1034.0,NW,242.70,0,0
43822,43823,2014,12,31,22,8.0,-22,-4.0,1034.0,NW,246.72,0,0


In [4]:
cbwd = pd.get_dummies(data['cbwd'])
cbwd.columns = ["cbwd_{}".format(i) for i in range(cbwd.shape[1])]
data = pd.concat([data, cbwd], axis=1)

In [5]:
data['cbwd']=data['cbwd'].factorize()[0]

In [6]:
data['pm2.5'] = data['pm2.5'].fillna(method='ffill').fillna(method='bfill')

In [7]:
data.drop(['cbwd_0','cbwd_1','cbwd_2','cbwd_3'], axis=1, inplace=True)

In [8]:
data

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,129.0,-21,-11.0,1021.0,0,1.79,0,0
1,2,2010,1,1,1,129.0,-21,-12.0,1020.0,0,4.92,0,0
2,3,2010,1,1,2,129.0,-21,-11.0,1019.0,0,6.71,0,0
3,4,2010,1,1,3,129.0,-21,-14.0,1019.0,0,9.84,0,0
4,5,2010,1,1,4,129.0,-20,-12.0,1018.0,0,12.97,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,43820,2014,12,31,19,8.0,-23,-2.0,1034.0,0,231.97,0,0
43820,43821,2014,12,31,20,10.0,-22,-3.0,1034.0,0,237.78,0,0
43821,43822,2014,12,31,21,10.0,-22,-3.0,1034.0,0,242.70,0,0
43822,43823,2014,12,31,22,8.0,-22,-4.0,1034.0,0,246.72,0,0


In [9]:
data.columns[5:]

Index(['pm2.5', 'DEWP', 'TEMP', 'PRES', 'cbwd', 'Iws', 'Is', 'Ir'], dtype='object')

In [10]:
from sklearn.preprocessing import MinMaxScaler
values = data[data.columns[5:]].values.reshape(-1,8)
values = values.astype('float32')
#scaler = MinMaxScaler(feature_range=(0, 1))
#scaled = scaler.fit_transform(values)
scaled = (values-(np.min(values)))/(np.max(values)-(np.min(values)))

In [11]:
train_size = int(len(scaled) * 0.8)
val_size = int(len(scaled) * 0.10)
test_size = len(scaled) - train_size-val_size
train, val, test = scaled[0:train_size,:],scaled[train_size:train_size+val_size,:], scaled[train_size+val_size:,:]
print(len(train),len(val), len(test))

35059 4382 4383


In [12]:
train_size = 35060
val_size = 4382
test_size = 4382

In [13]:
train, val, test = scaled[0:train_size,:],scaled[train_size:train_size+val_size,:], scaled[train_size+val_size:,:]
print(len(train),len(val), len(test))

35060 4382 4382


In [14]:
train_x = train[:,1:]
train_y = train[:,0]
val_x = val[:,1:]
val_y = val[:,0]
test_x = test[:,1:]
test_y = test[:,0]

In [15]:
print(train_x.shape)
print(train_y.shape)

(35060, 7)
(35060,)


## RMSE

In [16]:
depth_list = [1, 2, 3, 4, 5]
rmse_list = []

# Validation
for depth in depth_list:
    regressor = tree.DecisionTreeRegressor(max_depth=depth, min_samples_split=5) 
    regressor.fit(train_x, train_y)

    yhat = regressor.predict(val_x)
    rmse = sqrt(mean_squared_error(val_y, yhat))
    rmse_list.append(rmse)
    
    result = list(map(list, zip(depth_list, rmse_list)))
    result_df = DataFrame(result,columns=['max_depth', 'rmse'])

print('-------Validation-------')
print(result_df)

-------Validation-------
   max_depth      rmse
0          1  0.086982
1          2  0.078826
2          3  0.077608
3          4  0.076610
4          5  0.076103


In [17]:
# Test
best_depth = result_df.iloc[result_df['rmse'].argmin()]['max_depth']
regressor = tree.DecisionTreeRegressor(max_depth=best_depth, min_samples_split=5) 
regressor.fit(train_x, train_y)
predict = regressor.predict(test_x)
test_rmse = sqrt(mean_squared_error(test_y, predict))

print('Test RMSE :', test_rmse)

Test RMSE : 0.0723465302594698


# MAE

In [18]:
depth_list = [1, 2, 3, 4, 5]
mae_list = []

# Validation
for depth in depth_list:
    regressor = tree.DecisionTreeRegressor(max_depth=depth, min_samples_split=5) 
    regressor.fit(train_x, train_y)

    yhat = regressor.predict(val_x)
    mae = mean_absolute_error(val_y, yhat)
    mae_list.append(mae)
    
    result = list(map(list, zip(depth_list, mae_list)))
    result_df = DataFrame(result,columns=['max_depth', 'mae'])

print('-------Validation-------')
print(result_df)

-------Validation-------
   max_depth       mae
0          1  0.060600
1          2  0.054555
2          3  0.053527
3          4  0.052220
4          5  0.051263


In [19]:
# Test
best_depth = result_df.iloc[result_df['mae'].argmin()]['max_depth']
regressor = tree.DecisionTreeRegressor(max_depth=best_depth, min_samples_split=5) 
regressor.fit(train_x, train_y)
predicted = regressor.predict(test_x)
test_mae = mean_absolute_error(test_y, predicted)

print('Test MAE :', test_mae)

Test MAE : 0.05345453660955836
